In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import lstsq, solve
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torchvision.transforms as transforms
from PIL import Image, ImageOps

In [2]:
file_path = 'data/socal.csv'
df = pd.read_csv(file_path)

# Folder where the images are stored
image_folder = 'data/socal_pics'

# Create a new column 'image_path' by constructing the path based on 'image_id'
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(image_folder, f"{x}.jpg"))

# Display the first few rows of the data
df.head()

,image_id,street,citi,n_citi,bed,bath,sqft,price,image_path
0,0,1317 Van Buren Avenue,"Salton City, CA",317,3,2.0,1560,201900,data/socal_pics\0.jpg
1,1,124 C Street W,"Brawley, CA",48,3,2.0,713,228500,data/socal_pics\1.jpg
2,2,2304 Clark Road,"Imperial, CA",152,3,1.0,800,273950,data/socal_pics\2.jpg
3,3,755 Brawley Avenue,"Brawley, CA",48,3,1.0,1082,350000,data/socal_pics\3.jpg
4,4,2207 R Carrillo Court,"Calexico, CA",55,4,3.0,2547,385100,data/socal_pics\4.jpg


In [3]:
class RealEstateDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_id = self.data.iloc[idx]['image_id']
        image_path = f"{self.image_folder}/{image_id}.jpg"

        # Check if the image file exists
        if os.path.exists(image_path):
            image = Image.open(image_path).convert('RGB')
        else:
            print(f"Warning: Image {image_path} not found. Using a blank image.")
            image = Image.new('RGB', (64, 64), color=(255, 255, 255))  # Create a blank image

        if self.transform:
            image = self.transform(image)

        sqft = torch.tensor(self.data.iloc[idx]['sqft'], dtype=torch.float32)
        price = torch.tensor(self.data.iloc[idx]['price'], dtype=torch.float32)

        return image, sqft, price


In [4]:
class PricePredictor(nn.Module):
    def __init__(self):
        super(PricePredictor, self).__init__()

        # CNN for image processing, designed for 256x256 input images
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),  # Output: 128x128
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Output: 64x64

            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1), # Output: 32x32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Output: 16x16

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # Output: 8x8
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Output: 4x4

            nn.Flatten()
        )

        # Calculate the output size of the CNN by running a test tensor
        test_tensor = torch.randn(1, 3, 256, 256)  # Simulate a 256x256 input image
        cnn_output = self.cnn(test_tensor)
        self.cnn_output_size = cnn_output.shape[1]  # Dynamically get the flattened size

        # Fully connected layer for `sqft`
        self.fc_sqft = nn.Linear(1, 64)

        # Final layers for combined data
        self.fc_combined = nn.Sequential(
            nn.Linear(self.cnn_output_size + 64, 128),  # Combine CNN and sqft features
            nn.ReLU(),
            nn.Linear(128, 1)  # Output layer for price
        )

    def forward(self, image, sqft):
        # CNN part for image
        img_features = self.cnn(image)

        # Fully connected layer for sqft
        sqft_features = self.fc_sqft(sqft.view(-1, 1))

        # Concatenate both features
        combined_features = torch.cat((img_features, sqft_features), dim=1)

        # Pass through final layers
        price = self.fc_combined(combined_features)

        return price

In [5]:
# Define transform for 256x256 input images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.ToTensor()
])


# Initialize dataset and dataloader
csv_file = 'data/socal.csv'
image_folder = 'data/socal_pics'
dataset = RealEstateDataset(csv_file, image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = PricePredictor()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, sqft, prices in dataloader:
        # Forward pass
        predictions = model(images, sqft)
        loss = criterion(predictions.view(-1), prices)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 78756610048.0000
Epoch [2/10], Loss: 45900976128.0000
Epoch [3/10], Loss: 136016019456.0000
Epoch [4/10], Loss: 71221051392.0000
Epoch [5/10], Loss: 60624424960.0000
Epoch [6/10], Loss: 71590608896.0000
Epoch [7/10], Loss: 72826494976.0000
Epoch [8/10], Loss: 67494244352.0000
Epoch [9/10], Loss: 171699077120.0000
Epoch [10/10], Loss: 91075543040.0000


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from torchvision.models import VGG16_Weights


# Load the data
file_path = 'data/socal.csv'
df = pd.read_csv(file_path)

# Folder where the images are stored
image_folder = 'data/socal_pics'

# Create a new column 'image_path' by constructing the path based on 'image_id'
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(image_folder, f"{x}.jpg"))

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # VGG16 normalization
                         std=[0.229, 0.224, 0.225])
])

# Load pre-trained VGG16 model and modify it to output features
vgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT)
vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1])
vgg16.eval()

def extract_features(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return np.zeros(4096)  # Return a zero vector if image is not found or corrupted
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = vgg16(image)
    return features.numpy().flatten()

# Extract image features
image_features_list = []
for idx, row in df.iterrows():
    image_path = row['image_path']
    features = extract_features(image_path)
    image_features_list.append(features)

# Convert list to numpy array
image_features = np.array(image_features_list)

# Prepare numerical features
numerical_features = df[['bed', 'bath', 'sqft']].values

# Concatenate image features and numerical features
X = np.concatenate((image_features, numerical_features), axis=1)

# Target variable
y = df['price'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2 Score: {r2}")
